In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
if os.path.exists('incidentes_training.pkl'):
    print('Archivo recuperado')
    df_copy = pd.read_pickle('incidentes_training.pkl')
else:
    print('Archivo no encontrado')
    df_copy = pd.read_excel('compilado_delitos_2022.xlsx', dtype=str)
    df_copy.to_pickle('incidentes_training.pkl')

Archivo recuperado


In [4]:

df = df_copy.copy()
#Print dimensions of the dataset
print(df.shape)# Print columns of the dataset
print(df.columns)
# Print the first 5 rows of the data
# df.head()

(349804, 42)
Index(['folio', 'origen_incidente', 'fecha_creacion', 'hora_creacion',
       'dia_semana', 'fecha_cierre', 'hora_cierre', 'usuario_inicia',
       'existe_video', 'numero_llamante', 'nombre_llamante',
       'numero_alternativo', 'incidente_c4', 'tipo_intervencion_c2', 'calle',
       'numero', 'esquina', 'colonia', 'codigo_postal', 'delegacion_inicio',
       'sector_inicio', 'c2_inicio', 'latitud', 'longitud', 'senias',
       'codigo_cierre', 'comentarios', 'detenidos', 'tiempo_resp_fce',
       'clas_con_f_alarma', 'tipo_entrada', 'delegacion_cierre',
       'sector_cierre', 'c2_cierre', 'usuario_cierre', 'usuario_intervencion',
       'folio_padre', 't_aten_066', 't_desp_c2_c4', 't_unidad', 't_total',
       'inc-clas'],
      dtype='object')


In [5]:
''' Save columns and drop the ones that are not needed
       ['fecha_creacion', 'hora_creacion',
       'dia_semana', 'incidente_c4', 'colonia', 'delegacion_inicio',
       'sector_inicio', 'latitud', 'longitud']
'''
df.fillna('0', inplace=True)

# solo considerar los registros que tengan "A" en la columna codigo_cierre
df = df[df['codigo_cierre'] == 'A']

# Remove YYYY-MM-DD in field hora_creacion
df['hora_creacion'] = df['hora_creacion'].str[11:13]+':00'
# Remove rows with date 2021-12-31
df = df[df.fecha_creacion != '2021-12-31']
df = df[df.fecha_creacion != '2023-01-01']
# Remove YYYY of field fecha_creacion
# df['fecha_creacion'] = df['fecha_creacion'].str[5:]
# Map dia_semana from name to int-name
df['dia_semana'] = df['dia_semana'].replace({'Lunes':'1-Lunes', 'Martes':'2-Martes', 'Miércoles':'3-Miércoles', 'Jueves':'4-Jueves', 'Viernes':'5-Viernes', 'Sábado':'6-Sábado', 'Domingo':'7-Domingo'})
# Map mes_creacion from name to int-name
df['mes_creacion'] = df['fecha_creacion'].str[5:7].replace({'01':'01-Enero', '02':'02-Febrero', '03':'03-Marzo', '04':'04-Abril', '05':'05-Mayo', '06':'06-Junio', '07':'07-Julio', '08':'08-Agosto', '09':'09-Septiembre', '10':'10-Octubre', '11':'11-Noviembre', '12':'12-Diciembre'})
# Map dia-mes_creacion from name to int
df['dia_creacion'] = df['fecha_creacion'].str[8:]
# Get de week number of the year
df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'], format='%Y-%m-%d')
df['semana_creacion'] = df['fecha_creacion'].dt.isocalendar().week
# Parse str to datetime


columns = ['fecha_creacion', 'hora_creacion', 'mes_creacion', 'dia_creacion', 'semana_creacion',
        'dia_semana', 'incidente_c4', 'colonia', 'delegacion_inicio',
        'sector_inicio', 'latitud', 'longitud', 'codigo_cierre']

df = df[columns]



# Print the first 5 rows of the data
df.head()

,fecha_creacion,hora_creacion,mes_creacion,dia_creacion,semana_creacion,dia_semana,incidente_c4,colonia,delegacion_inicio,sector_inicio,latitud,longitud,codigo_cierre
0,2022-04-02,07:00,04-Abril,02,13,6-Sábado,Robo-Vehículo sin Violencia,ZONA ESCOLAR,GUSTAVO A. MADERO,CUAUTEPEC,19.540726,-99.1446619,A
2,2022-04-07,22:00,04-Abril,07,14,4-Jueves,Robo-Vehiculo con Violencia,SAN JUAN DE ARAGON VI SECCION,GUSTAVO A. MADERO,ARAGON,19.466062,-99.064584,A
3,2022-04-07,21:00,04-Abril,07,14,4-Jueves,Robo-Vehiculo con Violencia,LA PLANTA,IZTAPALAPA,TEZONCO,19.294715,-99.065944,A
4,2022-04-08,11:00,04-Abril,08,14,5-Viernes,Robo-Vehículo sin Violencia,EJIDOS DE SAN PEDRO MARTIR,TLALPAN,HUIPULCO-HOSPITALES,19.264921,-99.162166,A
6,2022-04-15,21:00,04-Abril,15,15,5-Viernes,Agresión-Persona,CENTRO (AREA 9),CUAUHTEMOC,CENTRO,19.42481,-99.132952,A


In [6]:
# diferentes codigo_cierre en el dataset
df.codigo_cierre.unique()

array(['A'], dtype=object)

In [7]:
#cuantos registros hay de cada codigo_cierre
df.codigo_cierre.value_counts()


A    127940
Name: codigo_cierre, dtype: int64

In [8]:
# Separar los datos por delitos
df_delitos = df.groupby('incidente_c4')
# df_delitos.head()

In [10]:
# 'Estadisticas trimestrales' en ingles: 'Trimestral statistics'
def statistic_frequency(df_main:pd.DataFrame, conditionals:list[pd.Series], groupby:str, titles:list[str], dir:str, num_high:int=5, num_low:int=5):
	from matplotlib.backends.backend_pdf import PdfPages
	import os

	if not os.path.exists(dir):
		os.makedirs(dir)

	with PdfPages(dir + '/'+ dir +'.pdf') as pdf:
		df_main_by_delitos = df_main.groupby('incidente_c4')
		for delito, delito_df in df_main_by_delitos:
			fig, axes = plt.subplots(nrows=1, ncols=len(conditionals), figsize=(5*len(conditionals), 5))
			# check if axes is a list or a single axis
			if not isinstance(axes, np.ndarray):
				axes = [axes]

			df_delito_pts = [delito_df[conditional] for conditional in conditionals]
			
			
			for df_section,title,axe in zip(df_delito_pts, titles, axes):
				delito_df_pt = df_section.groupby(groupby).count()['incidente_c4']
		
				delito_df_pt.plot(ax=axe, color='black', alpha=0.25)
				axe.set_title(title)
				axe.set_xticklabels(axe.get_xticklabels(), rotation=90)

				# Add scatter points with text labels to 5th hights days
				delito_df_pt1_sorted = delito_df_pt.sort_values(ascending=False)
				for i, (x, y) in enumerate(zip(delito_df_pt1_sorted.index, delito_df_pt1_sorted.values)):
					if i < num_low or i+num_high >= len(delito_df_pt1_sorted):
						x_pos = delito_df_pt.index.get_loc(x)
						axe.scatter(x_pos, y, s=10,  color='red' if i<num_low else 'green')
						axe.text(x_pos, y, str(f"{x} [{y}]"), fontsize=8, color='red' if i<num_low else 'green', rotation=90,  horizontalalignment='right', verticalalignment='top')

			#Get Max scale of all axes using get_ylim()
			y_max = max([ax.get_ylim()[1] for ax in axes])
			#Set all axes to the same scale
			for ax in axes:
				ax.set_ylim(0, y_max)


			plt.suptitle(delito)
			plt.savefig(f"{dir}/{delito}.png")
			pdf.savefig()
			plt.show()
		plt.close()

def statistic_heatmap(df_main:pd.DataFrame, index, columns, title, dir:str):
	import os
	import seaborn as sns
	import matplotlib.pyplot as plt
	from matplotlib.backends.backend_pdf import PdfPages

	if not os.path.exists(dir):
		os.makedirs(dir)


	with PdfPages(f'{dir}/{dir}.pdf') as pdf:
		df_main_by_delitos = df_main.groupby('incidente_c4')
		for delito, delito_df in df_main_by_delitos:
			# Create a pivot table with the count of incidents by day and hour
			delito_df_pt = delito_df.pivot_table(index=index, columns=columns, values='incidente_c4', aggfunc='count', fill_value=0)
			# Plot the heatmap
			fig, ax = plt.subplots()
			sns.heatmap(delito_df_pt, cmap='Spectral_r', annot=True, ax=ax, fmt='d')
			# plt.tick_params(axis='x', labelrotation=90, labeltop=True, labelbottom=False, labelsize=20)
			ax.set_title(title)
			plt.suptitle(delito)
			pdf.savefig()
			plt.savefig(f"{dir}/{delito}.png")
			plt.show()
		plt.close()

def statistic_geomap(df_main:pd.DataFrame, groupby:str, title:str, dir:str, USE_SECTORS:bool=True):
	import os
	import geopandas as gpd
	from shapely.geometry import Point
	from matplotlib.backends.backend_pdf import PdfPages
	import matplotlib.pyplot as plt
	import seaborn as sns

	if not os.path.exists(dir):
		os.makedirs(dir)

	if USE_SECTORS:
		mxcity = gpd.read_file('mapa_mexico_sectores/')\
			.set_index('TERRITORIA')\
			.to_crs(epsg=4326)
	else:
		# Obtain the map of Mexico
		mx = gpd.read_file('mapa_mexico/')\
				.set_index('CLAVE')\
				.to_crs(epsg=4326)
		# obtain the map of Mexico City with the geometry of the city
		mxcity = mx.query('CVE_EDO==\"09\"')

	# leer archivo xlsx de camaras in sheet 'base'
	df_cameras = pd.read_excel('BASE_24ENE023.xlsx', sheet_name='BASE')
	# ob
	df_cameras = df_cameras[['CYGW', 'CXGW']]
	df_cameras = df_cameras.dropna()

	# Coordinate reference system : WGS84
	crs = {'init': 'epsg:4326'}


	with PdfPages(f'{dir}/{dir}.pdf') as pdf:
		# Plot the cameras
		gdf_cameras = gpd.GeoDataFrame(
			df_cameras,
			crs=crs,
			geometry=gpd.points_from_xy(df_cameras.CXGW, df_cameras.CYGW)
		)
		gdf_cameras.plot(ax=ax, markersize=2, color='blue', alpha=0.1)

		plt.suptitle('Camaras')
		pdf.savefig()
		plt.savefig(f"{dir}/{delito}.png")
		plt.show()


		df_main_by_delitos = df_main.groupby('incidente_c4')
		for delito, delito_df in df_main_by_delitos:
			fig, ax = plt.subplots(figsize=(10, 10))

			mxcity.boundary.plot(ax=ax, color='black', linewidth=1.5)
			#mxcity.plot(ax=ax, color='white', edgecolor='black')

			# Create a GeoDataFrame from the DataFrame
			gdf = gpd.GeoDataFrame(
				delito_df, 
				crs=crs,
				geometry=gpd.points_from_xy(delito_df.longitud, delito_df.latitud)
			)

			# Plot the incidents
			gdf.plot(ax=ax, markersize=2, color='#26A337', alpha=0.075)

			# Add a title
			ax.set_title(title)
			plt.suptitle(delito)
			pdf.savefig()
			plt.savefig(f"{dir}/{delito}.png")
			plt.show()
		plt.close()

def statistic_heatgeomap(df_main:pd.DataFrame, index, columns, title, dir:str, USE_SECTORS:bool=True):
	import os
	import geopandas as gpd
	from shapely.geometry import Point
	from matplotlib.backends.backend_pdf import PdfPages
	import matplotlib.pyplot as plt
	import seaborn as sns

	if not os.path.exists(dir):
		os.makedirs(dir)

	if USE_SECTORS:
		mxcity = gpd.read_file('mapa_mexico_sectores/')\
			.set_index('TERRITORIA')\
			.to_crs(epsg=4326)
	else:
		# Obtain the map of Mexico
		mx = gpd.read_file('mapa_mexico/')\
				.set_index('CLAVE')\
				.to_crs(epsg=4326)
		# obtain the map of Mexico City with the geometry of the city
		mxcity = mx.query('CVE_EDO==\"09\"')

	# leer archivo csv camaras_sectoractual.csv
	df_cameras = pd.read_csv('CAMARAS_SECTORACTUAL.csv', sep=',')
	# ob
	df_cameras = df_cameras[['CYGW', 'CXGW']]
	df_cameras = df_cameras.dropna()

	# Coordinate reference system : WGS84
	crs = {'init': 'epsg:4326'}


	with PdfPages(f'{dir}/{dir}.pdf') as pdf:
		df_main_by_delitos = df_main.groupby('incidente_c4')
		for delito, delito_df in df_main_by_delitos:
			fig, ax = plt.subplots(figsize=(10, 10))

			mxcity.boundary.plot(ax=ax, color='black', linewidth=1.5)
			#mxcity.plot(ax=ax, color='white', edgecolor='black')

			# Create heatmap with dataframe in mexico city
			sns.heatmap(
				delito_df.pivot_table(index=index, columns=columns, aggfunc='size', fill_value=0),
				cmap='YlGnBu',
				ax=ax,
				linewidths=0.1,
				linecolor='white',
				annot=True,
				annot_kws={"size": 8},
				cbar=False
			)

			# Add a title
			ax.set_title(title)
			plt.suptitle(delito)
			pdf.savefig()
			plt.savefig(f"{dir}/{delito}.png")
			plt.show()
		plt.close()

In [8]:
# Contextual menu to ask for activity
def ask_for_activity():
	activity = input('''
		Elige una actividad:
		 1. Diagramas de frecuencia
		 2. Mapas de calor
		 3. Matrices de correlación
		 4. Matries de calor
		 5. Matries de calor por Delegación
	''')
	return activity

In [ ]:
# ask for activity
activity = ask_for_activity()

if activity == '1':
	# Tendencia por hora del dia
	statistic_frequency(
		df,
		[df['hora_creacion']>''],
		'hora_creacion',
		['Enero - Diciembre 2022'],
		'tendencia_diaria',
		4,6
	)

	# Tendencia por dia de la semana
	statistic_frequency(
		df,
		[df['incidente_c4']>''],
		'dia_semana',
		['Enero - Diciembre 2022'],
		'tendencia_semanal',
		2,2
	)

	## Tendencia por dia del mes
	# statistic_frequency(
	# 	df,
	# 	[df['incidente_c4']>''],
	# 	'dia_creacion',
	# 	['Enero - Diciembre 2022'],
	# 	'tendencia_mensual',
	# 	5,5
	# )

	# Tendencia por mes del año
	statistic_frequency(
		df,
		[
			df['fecha_creacion'] < '04-01',
			((df['fecha_creacion'] >= '04-01') & (df['fecha_creacion'] < '07-01')),
			((df['fecha_creacion'] >= '07-01') & (df['fecha_creacion'] < '10-01')),
			(df['fecha_creacion'] >= '10-01')
		], 
		'fecha_creacion', 
		['Enero - Marzo 2022', 'Abril - Junio 2022', 'Julio - Septiembre 2022', 'Octubre - Diciembre 2022'],
		'tendencia_trimestral',
		5, 5
	)
elif activity == '3':
	statistic_geomap(
	   df, 
	   'incidente_c4', 
	   'Enero - Diciembre 2022', 
	   'geomap_anual', 
	   True
	)
elif activity == '4':
	statistic_heatmap(
		df,
		'hora_creacion',
		'dia_semana',
		'Enero - Diciembre 2022',
		'heatmap_dia_hora',
	)
	statistic_heatmap(
		df,
		'dia_creacion',
		'mes_creacion',
		'Enero - Diciembre 2022',
		'heatmap_mes_dia',
	)
elif activity == '5':
	# Heatmap for each delegacion_inicio
	for delegacion in df['delegacion_inicio'].unique():
	    statistic_heatmap(
	        df[df['delegacion_inicio']==delegacion],
	        'hora_creacion',
	        'dia_semana',
	        f'{delegacion} | Enero - Diciembre 2022',
	        f'heatmap_dia_hora-{delegacion}',
	    )

In [ ]:
#statistic_heatgeomap(
#    df, 
#    'latitud', 'longitud',
#    'Enero - Diciembre 2022', 
#    'geomap_heat_anual', 
#    False
#)

In [16]:
df['incidente_c4'].unique()

array(['Robo-Vehículo sin Violencia', 'Robo-Vehiculo con Violencia',
       'Agresión-Persona', 'Denuncia-Persona Sospechosa',
       'Disturbio-Concentración de Personas', 'Disturbio-Disparos',
       'Robo-Auto partes', 'Robo-Automovilista',
       'Robo-Establecimiento con Violencia',
       'Robo-Establecimiento sin Violencia', 'Robo-Transeúnte',
       'Abandono-Vehículo'], dtype=object)

In [ ]:
# Install playwright with only chromium with npm
if not os.path.exists('node_modules') and not os.path.exists('node_modules/playwright'):
	!npm init -y
	!npm install playwright --save-dev
	!npx playwright install --with-deps chromium

import time
import folium
from folium.plugins import HeatMap
import io
from PIL import Image

# Create a heatmap for gruops of delitos
import os
if not os.path.exists('maps'):
	os.makedirs('maps')

for delito in df['incidente_c4'].unique():
    m = folium.Map(
		location=[19.325276250942087, -99.12435477661893],
        zoom_start=11,
        tiles='cartodbpositron',#'cartodbpositron'|'stamentoner'|'stamenterrain'|'stamenwatercolor'|'stamenTonerLabels'|'stamenTerrainLabels'|'stamenWatercolorLabels'|'cartodbpositronLabels'|'cartodbdark_matterLabels'|'cartodbpositronNoLabels'|'cartodbdark_matterNoLabels'
    )
	
    HeatMap(
        data=df[df['incidente_c4']==delito][['latitud', 'longitud']].values.tolist(),
        radius=15,
		use_local_extrema=True,
        max_zoom=13
    ).add_to(m)

    # open or create the folder with the name 'maps' to save the maps in html format 
    
    # save the map to html
    m.save(f'maps/{delito}-ule.html')
	# wait for the map to be saved

!node screenshot_maps.js


In [ ]:
# la funcion de coorelacion mas perra del world
def correlation(df, title):
    import seaborn as sns
    import matplotlib.pyplot as plt

    matriz=[]
    for delito in df:
        fila=[]
        for delito2 in df:
            if delito==delito2:
                fila.append(0)
            else:
                fila.append(1/(abs(delito-delito2)+1))
        matriz.append(fila)

    # graficar la matriz en un heatmap con los indices de los delitos
    plt.figure(figsize=(20,20))
    plt.title(title, fontsize=30, fontweight='bold')
    sns.heatmap(matriz, xticklabels=df.index, yticklabels=df.index, annot=True, cmap='Spectral_r')
    plt.tick_params(axis='x', labelrotation=90, labeltop=True, labelbottom=False)
    plt.show()
    return matriz

def relaciones_mayor_menor(matriz:list(list())):

In [ ]:
# Correlación de delitos en CDMX

delitos_count=df['incidente_c4'].value_counts()
matriz_correlacion = correlation(delitos_count, 'Correlacion entre delitos CDMX sin normalizar')
relaciones = relaciones_mayor_menor(matriz_correlacion)

# Normalización de datos, dónde 1 es el valor más alto
delitos_count_norm=delitos_count/delitos_count.max()
matriz_correlacion_norm = correlation(delitos_count_norm, 'Correlacion entre delitos CDMX normalizado')
relaciones_norm = relaciones_mayor_menor(matriz_correlacion_norm)

#create directory correlaciones
if not os.path.exists('correlaciones'):
	os.makedirs('correlaciones')

#Save it in excel
import pandas as pd
#join relaciones and relaciones_norm
relaciones = [relaciones[i]+relaciones_norm[i] for i in range(len(relaciones))]
df_cor = pd.DataFrame(relaciones, columns = ['Delito 1', 'Delito 2', 'Correlacion', 'Delito 1 Normalizado', 'Delito 2 Normalizado', 'Correlacion Normalizada'])
df_cor.to_excel(f'correlaciones/{delegacion}.xlsx', index=False)






In [ ]:
# Correlación de delitos en CDMX por delegacion
import seaborn as sns

# función para crear matriz de correcilación
def matriz_correlacion(delitos_count_norm):
	matriz=[]
	for delito in delitos_count_norm:
		fila=[]
		for delito2 in delitos_count_norm:
			if delito==delito2:
				fila.append(0)
			else:
				fila.append(1/(abs(delito-delito2)+1))
		matriz.append(fila)
	return matriz


df=df.dropna(subset=['delegacion_inicio'])
# print(df['delegacion_inicio'].unique())
for delegacion in df['delegacion_inicio'].unique():
    # filtrar por delegacion
    df_delegacion=df[df['delegacion_inicio']==delegacion]
    # contar los delitos
    delitos_count_norm=df_delegacion['incidente_c4'].value_counts()

    # Normalización de datos, dónde 1 es el valor más alto
    delitos_count_norm=delitos_count_norm/delitos_count_norm.max()

    # crear una matriz de correlacion
    matriz= matriz_correlacion(delitos_count_norm)
    # graficar la matriz en un heatmap con los indices de los delitos
    plt.figure(figsize=(20,20))
    plt.title(f'Correlacion entre delitos en {delegacion}', fontsize=30, fontweight='bold')
    sns.heatmap(matriz, xticklabels=delitos_count_norm.index, yticklabels=delitos_count_norm.index, annot=True, cmap='Spectral_r')
    plt.tick_params(axis='x', labelrotation=90, labeltop=True, labelbottom=False)
    plt.show()

    # Obtener los primeros 5 relaciones mas fuertes
    relaciones = []
    for i in range(len(matriz)):
        for j in range(i,len(matriz[i])):
            if i != j:
                relaciones.append([delitos_count_norm.index[i], delitos_count_norm.index[j], matriz[i][j]])
    relaciones.sort(key=lambda x: x[2], reverse=True)
    print(f'Las 5 relaciones mas fuertes en {delegacion} son:')
    for i in range(5):
        print(f'{relaciones[i][0]} - {relaciones[i][1]}: {relaciones[i][2]}')
    print()
    #creatédirectory correlaciones
    if not os.path.exists('correlaciones'):
        os.makedirs('correlaciones')

    #Save it in excel
    import pandas as pd
    df_cor = pd.DataFrame(relaciones[:5], columns = ['Delito 1', 'Delito 2', 'Correlacion'])
    df_cor.to_excel(f'correlaciones/{delegacion}.xlsx', index=False)



In [ ]:
# Análisis estádistico utilizando ARIMA

#importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# To install statsmodels use conda install -c conda-forge statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
# To install pmdarima use conda install -c conda-forge pmdarima
from pmdarima import auto_arima 

df_by_delitos = df.groupby('incidente_c4')

for delito, delito_df in df_by_delitos:
	print('delito', delito)

	delito_df_pt = delito_df.groupby('fecha_creacion').count()#['incidente_c4'].to_frame()
	delito_df_pt.rename(columns={'incidente_c4':'obs'}, inplace=True)
	delito_arima_df = delito_df_pt[['obs']]
	print('delito_df_pt', delito_arima_df)
	#print('columns', delito_df_pt.columns)

	#find if one day is missing and print it
	for i in range(len(delito_arima_df)-1):
		if delito_arima_df.index[i+1] - delito_arima_df.index[i] != pd.Timedelta(days=1):
			print('Missing day', delito_arima_df.index[i+1])
			# add missing day
			
			delito_arima_df.loc[delito_arima_df.index[i+1]] = 0
		

	# Create time series and plot
	RT = delito_arima_df.iloc[-678:].astype(float).squeeze().to_numpy()
	RT = pd.Series(RT, index=pd.date_range(start='2022-01-01', end='2022-12-31', freq='D'))
	plt.plot(RT)

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# To install statsmodels use conda install -c conda-forge statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
# To install pmdarima use conda install -c conda-forge pmdarima
from pmdarima import auto_arima 

# Read RDS file
echoT = pd.read_pickle("C:/Users/lpenac/Documents/SUPERBASES/echo_TOTAL.Rds")

# Filter and group data
RoboTranseunte = (
    echoT.loc[echoT['incidente_c4'] == 'Robo-Transeúnte']
        .groupby(['fecha_cierre', 'incidente_c4'])
        .count()
        .reset_index()
        .drop('incidente_c4', axis=1)
)

# Create date range and join with grouped data
date_range = pd.DataFrame({'fecha_cierre': pd.date_range(start='2021-01-01', end='2023-01-30', freq='D')})
Rob = pd.merge(date_range, RoboTranseunte, on='fecha_cierre', how='left')
Rob.fillna(0, inplace=True)
Rob.rename(columns={'n': 'obs'}, inplace=True)
Rob = Rob.drop('fecha_cierre', axis=1)

# Create time series and plot
RT = Rob.iloc[-678:].astype(float).squeeze().to_numpy()
RT = pd.Series(RT, index=pd.date_range(start='2021-03-01', end='2023-01-30', freq='D'))
plt.plot(RT)

# ADF test
result = adfuller(RT)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# Plot ACF and PACF
plot_acf(RT, lags=365)
plot_pacf(RT, lags=365)

# Model 1
model1 = ARIMA(RT, order=(1, 0, 1))
model1 = model1.fit()
print(model1.summary())
residuals = model1.resid

# Model 2
model2 = ARIMA(RT, order=(7, 0, 7))
model2 = model2.fit()
print(model2.summary())
residuals = model2.resid

# Forecast with Model 2
forecast = model2.forecast(steps=5)
print(forecast)

# Auto ARIMA
auto = auto_arima(RT, seasonal=False, suppress_warnings=True)
print(auto.summary())

# Model 3
model3 = ARIMA(RT, order=auto.order)
model3 = model3.fit()
print(model3.summary())
residuals = model3


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/lpenac/Documents/SUPERBASES/echo_TOTAL.Rds'

In [ ]:
# two points in crs epsg 4326, calculate their distance and if it is lower than 100mt print ok
pt1 = [19.42059, -99.14724]
pt2 = [19.41837, -99.14352]

# calculate distance
dist = geopy.distance.distance(pt1, pt2).m

# print distance
print(dist)

# if distance is lower than 100mt print ok
if dist < 100:
    print('ok')

In [17]:
# Load dataset "BASE_24ENE023.xlsx" with pandas
import pandas as pd
camaras_df_copy = pd.read_excel('BASE_24ENE023.xlsx', dtype=str, sheet_name='BASE') 


In [18]:
camaras_df = camaras_df_copy.copy()

In [19]:
"""ID_BCT_O,LATITUD,LONGITUD"""
# dropear las columnas que sean diferentes a "ID_BCT_O,LATITUD,LONGITUD" de camaras_df
camaras_df = camaras_df[['ID_BCT_O','LATITUD','LONGITUD']]
camaras_df

,ID_BCT_O,LATITUD,LONGITUD
0,1,19.435283,-99.147152
1,2,19.435098,-99.14582
2,3,19.434573,-99.143858
3,4,19.434374,-99.142704
4,5,19.434242,-99.14188
...,...,...,...
31318,MC18581,19.359499,-99.097748
31319,MC18749,19.445112,-99.123736
31320,MC17513,19.452434,-99.124443
31321,MC17609,19.435371,-99.119402


In [13]:
import geopy.distance
# to install geopy use 
# two points in crs epsg 4326, calculate their distance and if it is lower than 100mt print ok
pt1 = [19.42059, -99.14724]
pt2 = [19.41837, -99.14352]

# calculate distance
dist = geopy.distance.geodesic(pt1, pt2).m

# print distance
print(dist)

# if distance is lower than 100mt print ok
if dist < 100:
    print('ok')

461.55601385860615


In [20]:
# dropear colonia de df
df = df.drop('colonia', axis=1)
df.head()


,fecha_creacion,hora_creacion,mes_creacion,dia_creacion,semana_creacion,dia_semana,incidente_c4,delegacion_inicio,sector_inicio,latitud,longitud,codigo_cierre
0,2022-04-02,07:00,04-Abril,02,13,6-Sábado,Robo-Vehículo sin Violencia,GUSTAVO A. MADERO,CUAUTEPEC,19.540726,-99.1446619,A
2,2022-04-07,22:00,04-Abril,07,14,4-Jueves,Robo-Vehiculo con Violencia,GUSTAVO A. MADERO,ARAGON,19.466062,-99.064584,A
3,2022-04-07,21:00,04-Abril,07,14,4-Jueves,Robo-Vehiculo con Violencia,IZTAPALAPA,TEZONCO,19.294715,-99.065944,A
4,2022-04-08,11:00,04-Abril,08,14,5-Viernes,Robo-Vehículo sin Violencia,TLALPAN,HUIPULCO-HOSPITALES,19.264921,-99.162166,A
6,2022-04-15,21:00,04-Abril,15,15,5-Viernes,Agresión-Persona,CUAUHTEMOC,CENTRO,19.42481,-99.132952,A


In [22]:
import geopy
import geopy.distance
# To df order by latitud and longitud
df = df.sort_values(by=['latitud', 'longitud'])
df.latitud = df.latitud.astype(float)
df.longitud = df.longitud.astype(float)

#Parse camaras_df.LATITUD and cam.LONGITUD to float
camaras_df.LATITUD = camaras_df.LATITUD.astype(float)
camaras_df.LONGITUD = camaras_df.LONGITUD.astype(float)

# Find lower limit and upper limit of latitud and longitud where distance with each camera is lower than 100mt
for cam in camaras_df[:1].itertuples():
	# print(cam)
	print(f'camara: {cam.ID_BCT_O} <{cam.LATITUD:.10f},{cam.LONGITUD:.10f}>')
	pt1 = [cam.LATITUD, cam.LONGITUD]

	geopy.distance.geodesic([pt1[0],'0.0'], [df['latitud'],'0.0']).m

	# Find the index of the lower limit of latitud
	#latitud_lower_limit_index = df.iloc[geopy.distance.distance([pt1[0],'0.0'], [df['latitud'],'0.0']).meters < 100]
	#print(df[latitud_lower_limit_index])

camara: 1 <19.4352830000,-99.1471520000>


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [15]:
from geopy import distance
# obtener latitud y longitud de cada registro de camaras_df
for index, row in camaras_df.iterrows():
    print(index/len(camaras_df),"%", end="\r")
    lat1 = row['LATITUD']
    lon1 = row['LONGITUD']
    # obtener latitud y longitud de cada registro de df
        
    center = (lat1, lon1) # latitud y longitud del punto central
    # obtener la latitud y longitud de cada registro de df en una lista
    points = [(row['latitud'], row['longitud']) for index, row in df.iterrows()]

    radius = 50 # radio en metros

    # Calcular la distancia entre el punto central y cada punto
    distances = [distance.distance(center, point).m for point in points]

    # Filtrar los puntos dentro del radio
    within_radius = [point for point, dist in zip(points, distances) if dist <= radius]

    # crear una nueva matriz llamada puntos_etiquetados
    puntos_etiquetados = []
    
    # guardar en "puntos_etiquetados" todos los puntos dentro del radio copiar la informacion de todas las columnas de df y añadir una columna con el id de la camara
    for point in within_radius:
        punto = df.loc[(df['latitud'] == point[0]) & (df['longitud'] == point[1])]
        punto['id_camara'] = row['ID_BCT_O']
        puntos_etiquetados.append(punto)

puntos_etiquetados.head()

KeyboardInterrupt: 

In [21]:
import multiprocessing as mp

def process_camaras(row):
    
    lat1 = row['LATITUD']
    lon1 = row['LONGITUD']
    center = (lat1, lon1) # latitud y longitud del punto central
    # obtener la latitud y longitud de cada registro de df en una lista
    # convert next line in loop cycle
    points = [(row['latitud'], row['longitud']) for index, row in df.iterrows()]
    print(":)", end=' ')

    radius = 50 # radio en metros

    # Calcular la distancia entre el punto central y cada punto
    distances = [distance.distance(center, point).m for point in points]

    # Filtrar los puntos dentro del radio
    within_radius = [point for point, dist in zip(points, distances) if dist <= radius]

    puntos_etiquetados = []
    for point in within_radius:
        punto = df.loc[(df['latitud'] == point[0]) & (df['longitud'] == point[1])]
        punto['id_camara'] = row['ID_BCT_O']
        puntos_etiquetados.append(punto)

    return puntos_etiquetados

# crear una lista de argumentos para pasar a cada proceso
arguments = [(index, row) for index, row in camaras_df.iterrows()]

# iniciar los procesos y esperar a que terminen
with mp.Pool(processes=mp.cpu_count()) as pool:
    puntos_etiquetados = pool.starmap(process_camaras, arguments)

# concatenar los resultados en una sola DataFrame
puntos_etiquetados = pd.concat(puntos_etiquetados)
